In [15]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np

import numpy as np
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.pipeline import Pipeline
from sktime.transformations.panel.rocket import MiniRocketMultivariate

from drowsiness_detection import config
from drowsiness_detection.models import ThreeDStandardScaler
from drowsiness_detection.run_grid_search_experiment import load_experiment_data
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

config.set_paths(30, 10)



In [16]:
# load data
X_train, X_test, y_train, y_test, cv_splits = load_experiment_data(
    exclude_by="a",
    num_targets=2,
    seed=45,
    test_size=.2,
    split_by_subjects=True,
    use_dummy_data=False,
    nn_experiment=True,
    feature_col_indices=(5, 8, 9, 14, 15, 16, 19),
    model_name="mvts_transformer")

X_train shape: (28012, 300, 7), y_train shape: (28012,)
X_test shape: (7094, 300, 7), y_test shape: (7094,)


In [46]:
X_val = X_train[cv_splits == 1]
X_train = X_train[cv_splits == -1]
y_val = y_train[cv_splits == 1]
y_train = y_train[cv_splits==-1]


In [17]:
header = """
@problemName drowsiness-detection
@timeStamps false
@univariate false
@targetlabel true 0 1
@data

"""
def array_to_ts_file(data, labels, name):
    with open(f"{config.SOURCES_ROOT_PATH.parent}/data/ts_files/drowsiness_{name}.ts", "w") as fp:
        fp.write(header)
        for i in range(data.shape[0]):
            sample = data[i]
            np.savetxt(fp, sample.T, fmt='%.18f', delimiter=",", newline=":")
            fp.write(str(labels[i]))
            fp.write("\n")

In [18]:
%%time
array_to_ts_file(X_train, y_train, "TRAIN")
array_to_ts_file(X_test, y_test, "TEST")



CPU times: user 46.3 s, sys: 3.12 s, total: 49.4 s
Wall time: 50.9 s


In [48]:
array_to_ts_file(X_val, y_val, "VAL")


In [26]:
df, labels = load_from_tsfile_to_dataframe("drowsiness_TRAIN.ts", return_separate_X_and_y=True)


In [29]:
df.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6
0,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 0.000000 1 -15.008540 2 -11.25...,0 0.000000 1 8.697372 2 1.24...,0 643.000000 1 640.200012 2 641...,0 511.899994 1 513.700012 2 513...,0 0.429663 1 0.426461 2 0.42437...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...
1,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 7.504267 1 18.760635 2 12.50...,0 -16.149847 1 7.453803 2 7.45...,0 641.200012 1 643.299988 2 643...,0 540.599976 1 538.599976 2 538...,0 0.396403 1 0.391243 2 0.40187...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...
2,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 7.502229 1 -5.001406 2 7.50...,0 -2.484224 1 4.968349 2 4.96...,0 601.799988 1 601.299988 2 603...,0 471.600006 1 470.600006 2 469...,0 0.353105 1 0.352039 2 0.35167...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...
3,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 6.252714 1 0.000000 2 22.51...,0 -26.090376 1 -24.848209 2 -17.39...,0 663.599976 1 664.299988 2 662...,0 493.600006 1 492.399994 2 495...,0 0.336619 1 0.335555 2 0.32910...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...
4,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 -12.507121 1 -10.005696 2 7.50...,0 -17.379965 1 -16.138313 2 -19.86...,0 638.400024 1 640.099976 2 639...,0 444.200012 1 441.899994 2 447...,0 0.323776 1 0.324087 2 0.32386...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...


In [33]:
labels.shape


(20563,)

In [50]:
from mvts_transformer.src.datasets.utils import load_from_tsfile_to_dataframe


In [54]:
df, labels = load_from_tsfile_to_dataframe(f"{config.SOURCES_ROOT_PATH.parent}/data/ts_files/drowsiness_VAL.ts", return_separate_X_and_y=True)


8it [04:10, 31.36s/it]


KeyboardInterrupt: 

In [58]:
df.iloc[0,2]

0        0.000000
1       -2.484967
2     -232.199539
3       57.488182
4      170.392410
          ...    
295      0.000000
296      0.000000
297      0.000000
298      0.000000
299      0.000000
Length: 300, dtype: float64

In [4]:
from mvts_transformer.src.datasets.data import TSRegressionArchive

In [10]:
exp_config = {
    "task" : "classification",
    "subsample_factor": 0
}
data_loader = TSRegressionArchive(root_dir=f"{config.SOURCES_ROOT_PATH.parent}/data/ts_files/", pattern="VAL",config=exp_config)

7456it [00:14, 529.41it/s]


In [14]:
data_loader.all_df


,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6
0,0.0,0.000000,0.000000,637.700012,510.600006,0.287283,0.0
0,0.0,1.250712,-2.484967,639.299988,513.200012,0.301618,0.0
0,0.0,-6.253456,-232.199539,629.799988,567.799988,0.279901,0.0
0,0.0,30.015783,57.488182,655.700012,958.599976,0.453259,0.0
0,0.0,15.008442,170.392410,633.900024,695.400024,0.513857,0.0
...,...,...,...,...,...,...,...
7448,0.0,6.225692,11.149714,486.600006,386.600006,0.059951,0.0
7448,0.0,-2.490109,-2.478120,485.200012,390.100006,0.053074,0.0
7448,0.0,23.654440,-6.196422,483.399994,394.399994,0.041793,0.0
7448,0.0,34.862957,-29.748281,485.500000,398.000000,0.046397,0.0
